# Simulation of a cart pendulum

## The model
First of all it's worthwhile to design a non linear function of the cart-pendulum derived from $\eqref{eq:set-diff-eq}$.
Then, it's used the m-code to check a situation when the pendulum is in the "up" position (unstable equilibrium) and the initial condition is $y[t=0] = [x=0;\dot x=0;\theta=pi;\dot \theta=.5]$. It would be produced a periodic oscillation of $\theta$ because acceleration of pendulum put it out of its position of equilibrium (unstable). Think the cart pendulum over a railway and the pendulum-leg can rotate over its up down position freely without falling down a physical ground. Multi-oscillation are dumped for the cart-friction represented by a "d" parameter.

So the function cartpend define the unlinear differential equations:
```
function dy = cartpend(y,m,M,L,g,d,u)

Sy = sin(y(3));
Cy = cos(y(3));
D = m*L*L*(M+m*(1-Cy^2));

dy(1,1) = y(2);
dy(2,1) = (1/D)*(-m^2*L^2*g*Cy*Sy + m*L^2*(m*L*y(4)^2*Sy - d*y(2))) + 
m*L*L*(1/D)*u;
dy(3,1) = y(4);
dy(4,1) = (1/D)*((m+M)*m*g*L*Sy - m*L*Cy*(m*L*y(4)^2*Sy - d*y(2))) - 
m*L*Cy*(1/D)*u +.01*randn;
```
And the rest of the code to plot the variables evolution

```
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

tspan = 0:.1:30;
% pendulum in up positoin omega = .5
y0 = [0; 0; pi; .5];
% solve ode
[t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,0),tspan,y0);
% plot theta(t)
plot(t,y(:,3))
```

|          |     
|:----------:|
|![thetapen\label{thetapend}](images/thetapend.png)|

<p style="text-alignment=left;"><a name="fig2">Fig. 2</a> - Imposing boundary condition to put the pendulum out of its unstable equilibrium and showing the evolution </p>

## The modal form and the meaning of its eigenvalues

Like shown in the <a href="#section_1_2">chapter 2.2</a> the modal matrices form is really useful to shown poles over the its diagonal. Matlab gives the possibility to transform the initial set into the desired modal by the command *canon*. T is the transformation between these two representation that using different state variables describe the same physical system. The m-code:

```
s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];
H = [1 1 1 1];
J = 0;
sys = ss(F,G,H,J);
[modal_sys,T] = canon(sys, 'modal')

modal_sys =
 
  A = 
            x1       x2       x3       x4
   x1        0        0        0        0
   x2        0    2.434        0        0
   x3        0        0   -2.467        0
   x4        0        0        0  -0.1665
 
  B = 
            u1
   x1        1
   x2   0.2424
   x3  -0.2807
   x4   -1.052
 
  C = 
              x1         x2         x3         x4
   y1          1    0.02457    0.02786     0.9486
   y2          0    0.05981   -0.06874    -0.1579
   y3          0    0.07876    0.07793  -0.002645
   y4          0     0.1917    -0.1923  0.0004404
 
  D = 
       u1
   y1   0
   y2   0
   y3   0
   y4   0
 
Continuous-time state-space model.


T =

    1.0000    6.0000   -0.0000   -2.0000
         0   -0.0996    6.3858    2.6237
         0   -0.1138    6.3656   -2.5799
         0   -6.3195   -0.3524    2.1163
```
A note about $\pmb A$ matrix; it's known that poles on the right half-plane means unstable systems. It's a confirmation of instability when the pendulum is in the top position (look at the <a href="#fig2">figure 2</a> $\theta$ when the pendulum comes out of its unstable position i.e. either few cents of degree off its top position or like the simulation $\omega = 0.5$) 

It turns out that the transformation matrix that will convert the state description matrices to modal form has its columns the eigenvectors of **F**. The eigenvalues of cart-pendulum are directly findable solving for **F**
```
>> eig(F)

ans =

         0
   -2.4674
   -0.1665
    2.4339
```
Backing to $\eqref{eq:C}$ it's time to check the controllability of the card pendulum system:
```
>> ctrb(A,B)

ans =

         0    0.2000   -0.0400    0.2080
    0.2000   -0.0400    0.2080   -0.0816
         0    0.1000   -0.0200    0.6040
    0.1000   -0.0200    0.6040   -0.1408

>> rank(ctrb(A,B))

ans =

     4
```
The rank of $\mathcal C$ is full and it's possible to control the **full state system** by looping **x**.  